#Empezamos el proceso de ETL importando las librerías necesarias

In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import gzip
import io
import dask.dataframe as dd
import pyarrow as pa
import json
import re
import pyarrow.parquet as pq  # Importamos Parquet

Llamamos al primer dataset, que se encuentra en formato GNU Zip, un formato excelente para comprimir grandes cantidades de datos. Por lo tanto, es preferible usarlos sin descomprimirlos. Para ello usamos la librería GZIP. 

In [107]:
df = dd.read_json('Datasets\steam_games.json.gz', compression='gzip') #Usamos la librería DASK para llamar al json sin descomprimirlo
pdf = df.compute() #Utilizamos la función compute de DASK para pasar todo a un DATAFRAME de PANDAS
print(pdf.shape) #Imprimimos la cantidad de Filas y Columnas de nuestro DATAFRAME
print(pdf.head()) #Imprimimos las primeros 5 lineas de nuestro DATAFRAME

(120445, 13)
  publisher genres app_name title   url release_date  tags reviews_url specs  \
0      <NA>   <NA>     <NA>  <NA>  <NA>         <NA>  <NA>        <NA>  <NA>   
1      <NA>   <NA>     <NA>  <NA>  <NA>         <NA>  <NA>        <NA>  <NA>   
2      <NA>   <NA>     <NA>  <NA>  <NA>         <NA>  <NA>        <NA>  <NA>   
3      <NA>   <NA>     <NA>  <NA>  <NA>         <NA>  <NA>        <NA>  <NA>   
4      <NA>   <NA>     <NA>  <NA>  <NA>         <NA>  <NA>        <NA>  <NA>   

  price  early_access  id developer  
0  <NA>           NaN NaN      <NA>  
1  <NA>           NaN NaN      <NA>  
2  <NA>           NaN NaN      <NA>  
3  <NA>           NaN NaN      <NA>  
4  <NA>           NaN NaN      <NA>  


Como se puede apreciar, los datos proporcionados están llenos de nulos, por lo cual debemos empezar la limpieza.
Para ello, utilizaremos la función InfoDF, que nos proporcionará información de valor sobre el DATAFRAME.

In [109]:
def infoDF(pdf):
    # Cantidad de filas y columnas
    cantidad_filas = pdf.shape[0]
    cantidad_columnas = pdf.shape[1]
    
    # Cantidad de filas con datos nulos
    filas_con_nulos = pdf.isnull().any(axis=1).sum()
    
    # Porcentaje de nulos por fila y por columna
    porcentaje_nulos_filas = (pdf.isnull().sum(axis=1) / cantidad_columnas * 100).round(2)
    porcentaje_nulos_columnas = (pdf.isnull().sum() / cantidad_filas * 100).round(2)
    
    # Información general
    print("Información general del DataFrame:")
    print(f"Cantidad de filas: {cantidad_filas}")
    print(f"Cantidad de columnas: {cantidad_columnas}")
    print(f"Cantidad de filas con datos nulos: {filas_con_nulos}")
    
    # Porcentaje de nulos por fila
    print("\nPorcentaje de datos nulos por fila:")
    print(porcentaje_nulos_filas)
    
    # Porcentaje de nulos por columna
    print("\nPorcentaje de datos nulos por columna:")
    print(porcentaje_nulos_columnas)

infoDF(pdf)

Información general del DataFrame:
Cantidad de filas: 120445
Cantidad de columnas: 13
Cantidad de filas con datos nulos: 97915

Porcentaje de datos nulos por fila:
0         100.00
1         100.00
2         100.00
3         100.00
4         100.00
           ...  
120440      0.00
120441      0.00
120442      0.00
120443      0.00
120444     38.46
Length: 120445, dtype: float64

Porcentaje de datos nulos por columna:
publisher       80.00
genres          76.05
app_name        73.32
title           75.02
url             73.32
release_date    75.04
tags            73.46
reviews_url     73.32
specs           73.88
price           74.46
early_access    73.32
id              73.32
developer       76.06
dtype: float64


Ahora, descartaremos todas las filas dentro del DATAFRAME, que contengan puros datos nulos

In [110]:
pdf = pdf.dropna(how='all').copy() #Borramos las filas con nulos y las reasignamos al dataframe
infoDF(pdf) #Volvemos a utilizar la función infoDF para volver a revisar los datos

Información general del DataFrame:
Cantidad de filas: 32135
Cantidad de columnas: 13
Cantidad de filas con datos nulos: 9605

Porcentaje de datos nulos por fila:
88310      0.00
88311      0.00
88312      0.00
88313      0.00
88314     38.46
          ...  
120440     0.00
120441     0.00
120442     0.00
120443     0.00
120444    38.46
Length: 32135, dtype: float64

Porcentaje de datos nulos por columna:
publisher       25.06
genres          10.22
app_name         0.01
title            6.38
url              0.00
release_date     6.43
tags             0.51
reviews_url      0.01
specs            2.08
price            4.29
early_access     0.00
id               0.01
developer       10.27
dtype: float64


Para un mejor análisis, creamos un nuevo DATAFRAME que contenga únicamente filas donde exista al menos UN (1) dato nulo

In [111]:
def crear_dataframe_nulos(pdf):
    # Identificar filas y columnas con datos nulos
    filas_con_nulos = pdf[pdf.isnull().any(axis=1)]
    columnas_con_nulos = pdf.loc[:, pdf.isnull().any()]
    
    # Crear un nuevo DataFrame con las filas y columnas que tienen datos nulos
    df_nulos = pd.concat([filas_con_nulos, columnas_con_nulos], axis=1)
    
    return df_nulos
df_nulos = crear_dataframe_nulos(pdf)

Luego de eliminar las filas que contenían nulos, vamos a revisar que no queden filas duplicadas

In [112]:
def verificar_filas_duplicadas(pdf):
    # Verificar si hay filas duplicadas
    hay_duplicados = pdf.duplicated().any()
    
    if hay_duplicados:
        print("Se encontraron filas duplicadas en el DataFrame.")
    else:
        print("No se encontraron filas duplicadas en el DataFrame.")

print(verificar_filas_duplicadas(pdf))

No se encontraron filas duplicadas en el DataFrame.
None


Parece que por ahora viene todo en orden, para continuar con la limpieza de los datos, debemos corroborar en qué tipo se encuentra cada columna. 

In [113]:
pdf.dtypes


publisher       string[pyarrow]
genres          string[pyarrow]
app_name        string[pyarrow]
title           string[pyarrow]
url             string[pyarrow]
release_date    string[pyarrow]
tags            string[pyarrow]
reviews_url     string[pyarrow]
specs           string[pyarrow]
price           string[pyarrow]
early_access            float64
id                      float64
developer       string[pyarrow]
dtype: object

Para la primera función de este proyecto, necesitamos 3 columnas cruciales: Price, Developer, Release_Date

Primero, acomodaremos la columna price, para eliminar todos sus datos nulos y corregir sus valores a tipo float

In [114]:
pdf = pdf.dropna(subset=['price'])
pdf.loc[:, 'price'] = pdf['price'].apply(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == pd.to_numeric(x, errors='coerce') else 0)
pdf.dtypes

C:\Users\loren\AppData\Local\Temp\ipykernel_18864\1506138911.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[4.99 0.   0.   ... 1.99 4.99 4.99]' has dtype incompatible with string, please explicitly cast to a compatible dtype first.
  pdf.loc[:, 'price'] = pdf['price'].apply(lambda x: float(x) if pd.to_numeric(x, errors='coerce') == pd.to_numeric(x, errors='coerce') else 0)


publisher       string[pyarrow]
genres          string[pyarrow]
app_name        string[pyarrow]
title           string[pyarrow]
url             string[pyarrow]
release_date    string[pyarrow]
tags            string[pyarrow]
reviews_url     string[pyarrow]
specs           string[pyarrow]
price                   float64
early_access            float64
id                      float64
developer       string[pyarrow]
dtype: object

Una vez solucionada la columna price, pasamos a solucionar la columna release_date

Siguiendo la misma lógica, eliminamos las filas donde release_date tenga datos nulos, y analizamos si todas las fechas están en formato YYYY-MM-DD

In [115]:
pdf = pdf.dropna(subset=['release_date'])
# Función para identificar valores que no coinciden con el formato especificado
def encontrar_valores_incorrectos(pdf):
    try:
        pd.to_datetime(pdf['release_date'], format='%Y-%m-%d', errors='raise')
    except ValueError as e:
        valores_incorrectos = pdf.loc[pd.to_datetime(pdf['release_date'], errors='coerce').isnull(), 'release_date']
        return valores_incorrectos, str(e)
# Identificar valores que no coinciden con el formato especificado
valores_incorrectos, error_message = encontrar_valores_incorrectos(pdf)

if valores_incorrectos is not None:
    print("\nValores incorrectos en la columna 'release_date':")
    print(valores_incorrectos)
else:
    print("\nNo se encontraron valores incorrectos en la columna 'release_date'.")

if error_message:
    print("\nMensaje de error:")
    print(error_message)

ver_valores = encontrar_valores_incorrectos(pdf)


Valores incorrectos en la columna 'release_date':
88560        Jun 2009
88816        Oct 2010
88819        Oct 2010
88820        Oct 2010
88909        Feb 2011
90917        Sep 2014
91814        Apr 2015
94314        Apr 2016
94954        Jul 2016
98873        Jul 2017
101344           SOON
103983           2018
104953       Jul 2017
106530       Apr 2017
107790       Jan 2017
108995       Nov 2016
109065       Nov 2016
109456       Oct 2016
110678       Jul 2016
111237       Jun 2016
114031       Aug 2015
114785       Jun 2015
114960       May 2015
115590       Feb 2015
115966       Jan 2015
116428       Nov 2014
116786       Aug 2014
117096       Jul 2014
117324       May 2014
118632       Feb 2013
118781       Dec 2012
119742       Jul 2010
119784       Mar 2010
119826       Jan 2010
119879       Oct 2009
119928       Sep 2009
119980       Jun 2009
120268    coming soon
120318          SOON™
Name: release_date, dtype: string

Mensaje de error:
time data "Jun 2009" doesn't match for

Como vemos, hay valores que no corresponden al formato YYYY-MM-DD y los debemos tratar por separado

In [116]:
# Función para extraer el año
def extraer_ano(fecha):
    # Intentamos extraer el año del formato 'YYYY-MM-DD'
    try:
        return int(fecha.split('-')[0])
    except:
        pass
    
    # Intentamos extraer el año del formato 'MMM YYYY'
    try:
        return int(fecha.split()[-1])
    except:
        pass
    
    # Si no podemos extraer el año, devolvemos 0
    return 0

# Aplicamos la función a la columna 'release_date'
pdf['release_date'] = pdf['release_date'].apply(extraer_ano)

Ahora, como podemos ver a continuación, la columna price y la columna release_date se encuentran en formato numérico, para un mejor manejo

In [117]:
pdf.dtypes

publisher       string[pyarrow]
genres          string[pyarrow]
app_name        string[pyarrow]
title           string[pyarrow]
url             string[pyarrow]
release_date              int64
tags            string[pyarrow]
reviews_url     string[pyarrow]
specs           string[pyarrow]
price                   float64
early_access            float64
id                      float64
developer       string[pyarrow]
dtype: object

Para continuar el proceso de limpieza, debemos tratar de unificar las columnas genres y tags, ya que, como podemos ver a continuación, las mismas los datos son muy similares y se complementan.

In [118]:
pdf[['genres', 'tags']]

,genres,tags
88310,"['Action', 'Casual', 'Indie', 'Simulation', 'S...","['Strategy', 'Action', 'Indie', 'Casual', 'Sim..."
88311,"['Free to Play', 'Indie', 'RPG', 'Strategy']","['Free to Play', 'Strategy', 'Indie', 'RPG', '..."
88312,"['Casual', 'Free to Play', 'Indie', 'Simulatio...","['Free to Play', 'Simulation', 'Sports', 'Casu..."
88313,"['Action', 'Adventure', 'Casual']","['Action', 'Adventure', 'Casual']"
88315,"['Action', 'Adventure', 'Simulation']","['Action', 'Adventure', 'Simulation', 'FPS', '..."
...,...,...
120439,"['Action', 'Adventure', 'Casual', 'Indie']","['Action', 'Indie', 'Casual', 'Violent', 'Adve..."
120440,"['Casual', 'Indie', 'Simulation', 'Strategy']","['Strategy', 'Indie', 'Casual', 'Simulation']"
120441,"['Casual', 'Indie', 'Strategy']","['Strategy', 'Indie', 'Casual']"
120442,"['Indie', 'Racing', 'Simulation']","['Indie', 'Simulation', 'Racing']"


In [119]:
# Reemplazar los valores nulos en la columna 'genres' con una cadena vacía
pdf['genres'].fillna('', inplace=True)

# Reemplazar los valores nulos en la columna 'tags' con una cadena vacía
pdf['tags'].fillna('', inplace=True)

# Función para combinar los valores de 'genres' y 'tags'
def combine_genres_tags(row):
    genres = row['genres']
    tags = row['tags']
    
    # Si 'genres' está vacío, asigna los valores de 'tags'
    if not genres:
        return tags
    
    # Si 'tags' tiene más información que 'genres', combina ambas
    if tags:
        tags_list = tags.split(', ')
        for tag in tags_list:
            if tag not in genres:
                genres += ', ' + tag
    return genres

# Aplicar la función a cada fila del DataFrame
pdf['genres'] = pdf.apply(combine_genres_tags, axis=1)


C:\Users\loren\AppData\Local\Temp\ipykernel_18864\3010365759.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pdf['genres'].fillna('', inplace=True)
C:\Users\loren\AppData\Local\Temp\ipykernel_18864\3010365759.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

Con esto, ya podemos solucionar el primer ENDPOINT solicitado, para el cual necesitábamos ingresar un string con un Developer, y nos arrojará el porcentaje de juegos gratuitos que salieron cada año

In [120]:

table = pa.Table.from_pandas(pdf)
# Escribimos las tabla en un archivo Parquet
pq.write_table(table, 'Datasets/pdf_SteamGames.parquet')


Vamos con el segundo DATAFRAME, en este caso, Steam Games. El mismo, tiene errores de escritura por lo cual debemos cambiar el proceso de carga de datos.

In [121]:
import ast

#Creamos una lista vacía llamada "rows" donde almacenaremos los datos del archivo JSON.
rows = []
#Abrir el archivo "user_reviews.json/australian_user_reviews.json" con la codificación MacRoman.
with open(r"Datasets\user_reviews.json", encoding='utf-8') as f:
    # Leer cada línea del archivo.
    for line in f.readlines():
        # Utilizar "ast.literal_eval" para convertir cada línea en un diccionario de Python
        # y agregarlo a la lista "rows".
        rows.append(ast.literal_eval(line))

#Crear un DataFrame de Pandas a partir de la lista de diccionarios "rows".
df_user_reviews = pd.DataFrame(rows)
#Veamos unos registros al asar
df_user_reviews.sample(2)

,user_id,user_url,reviews
7567,dmckay,http://steamcommunity.com/id/dmckay,"[{'funny': '', 'posted': 'Posted January 2.', ..."
22074,76561198069625198,http://steamcommunity.com/profiles/76561198069...,"[{'funny': '', 'posted': 'Posted October 27, 2..."


Repetimos función para analizar el estado del Dataframe

In [122]:
infoDF(df_user_reviews)

Información general del DataFrame:
Cantidad de filas: 25799
Cantidad de columnas: 3
Cantidad de filas con datos nulos: 0

Porcentaje de datos nulos por fila:
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
25794    0.0
25795    0.0
25796    0.0
25797    0.0
25798    0.0
Length: 25799, dtype: float64

Porcentaje de datos nulos por columna:
user_id     0.0
user_url    0.0
reviews     0.0
dtype: float64


Este dataframe no contiene datos nulos pero nos encontramos con un nuevo desafío. La fila de reviews es una cantidad de listas anidadas que debemos desanidar

In [124]:
print("Columna reviews:")
print(df_user_reviews['reviews'].iloc[0])  # Imprimir un valor específico de la columna 'reviews'
print(df_user_reviews['reviews'].apply(type).unique())  # Verificar el tipo de dato de toda la columna 'reviews'

Columna reviews:
[{'funny': '', 'posted': 'Posted November 5, 2011.', 'last_edited': '', 'item_id': '1250', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'}, {'funny': '', 'posted': 'Posted July 15, 2011.', 'last_edited': '', 'item_id': '22200', 'helpful': 'No ratings yet', 'recommend': True, 'review': "It's unique and worth a playthrough."}, {'funny': '', 'posted': 'Posted April 21, 2011.', 'last_edited': '', 'item_id': '43110', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]
[<class 'list'>]


Para que nuestro Dataframe tenga sentido, debemos desarmar esta lista y crear columnas por separado

In [125]:
# Crear una lista vacía para almacenar las nuevas filas
new_rows = []

# Iterar sobre cada fila del DataFrame original
for index, row in df_user_reviews.iterrows():
    # Obtener la lista de reviews de la fila actual
    reviews_list = row.get('reviews')
    # Verificar si la lista de reviews está presente y no es None
    if reviews_list is not None:
        # Iterar sobre cada review en la lista
        for review_dict in reviews_list:
            # Crear una nueva fila con los datos de la review y otros datos de la fila original
            new_row = {
                'user_id': row['user_id'],
                'user_url': row['user_url'],
                'funny': review_dict.get('funny'),
                'posted': review_dict.get('posted'),
                'last_edited': review_dict.get('last_edited'),
                'item_id': review_dict.get('item_id'),
                'helpful': review_dict.get('helpful'),
                'recommend': review_dict.get('recommend'),
                'review': review_dict.get('review')
            }
            # Agregar la nueva fila a la lista de filas
            new_rows.append(new_row)

# Crear un nuevo DataFrame con las nuevas filas
new_df_user_reviews = pd.DataFrame(new_rows)

# Ver el nuevo DataFrame
print(new_df_user_reviews)

                 user_id                                           user_url  \
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
2      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
3                js41637               http://steamcommunity.com/id/js41637   
4                js41637               http://steamcommunity.com/id/js41637   
...                  ...                                                ...   
59300  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
59301  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
59302        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
59303        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
59304        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   

                                  funny            

Mucho mejor, ahora corroboramos que no existan filas duplicadas

In [126]:
filas_duplicadas = new_df_user_reviews[new_df_user_reviews.duplicated()]
num_filas_duplicadas = len(filas_duplicadas)
if num_filas_duplicadas > 0:
    print("Se encontraron", num_filas_duplicadas, "filas duplicadas en el DataFrame.")
else:
    print("No se encontraron filas duplicadas en el DataFrame.")

Se encontraron 874 filas duplicadas en el DataFrame.


Se encontraron muchas, asi que toca eliminarlas.

In [127]:
# Eliminar filas duplicadas en todo el DataFrame
new_df_user_reviews_sin_duplicados = new_df_user_reviews.drop_duplicates()

Todo listo, ahora eliminamos las columnas que no tienen valor para nuestro futuro análisis.

In [131]:
# Eliminar columnas usando el método drop()
columnas_a_eliminar = ['funny', 'last_edited', 'helpful']
new_df_user_reviews_sin_duplicados.drop(columnas_a_eliminar, axis=1, inplace=True)

# Verificar que las columnas se han eliminado
print(new_df_user_reviews_sin_duplicados.columns)

KeyError: "['funny', 'last_edited', 'helpful'] not found in axis"

Y empezamos la corrección de tipos de datos


In [133]:
print(new_df_user_reviews_sin_duplicados.dtypes)

user_id      object
user_url     object
posted       object
item_id      object
recommend      bool
review       object
dtype: object


Lo primero que debemos corregir es la columna 'posted', ya que la misma no coincide dentro de los datos y nos complicaría el funcionamiento de la función. Para ello crearemos una nueva columna 'year' donde extraeremos únicamente el año en que fue publicada la reseña.

In [134]:
# Función para extraer el año de la columna 'posted'
def extract_year(date_string):
    # Expresión regular para encontrar el año
    year_pattern = r'\b\d{4}\b'
    # Buscar el año en el texto
    year_match = re.search(year_pattern, date_string)
    if year_match:
        return int(year_match.group())
    else:
        return None

# Aplicar la función para extraer el año y crear una nueva columna 'year'
new_df_user_reviews_sin_duplicados['year'] = new_df_user_reviews_sin_duplicados['posted'].apply(extract_year)

# Calcular el promedio de los años presentes en el dataframe
average_year = new_df_user_reviews_sin_duplicados['year'].mean()

# Rellenar los valores de año faltantes con el promedio
new_df_user_reviews_sin_duplicados['year'].fillna(average_year, inplace=True)

# Convertir los años a enteros
new_df_user_reviews_sin_duplicados['year'] = new_df_user_reviews_sin_duplicados['year'].astype(int)

# Imprimir el dataframe resultante
print(new_df_user_reviews_sin_duplicados)

                 user_id                                           user_url  \
0      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
2      76561197970982479  http://steamcommunity.com/profiles/76561197970...   
3                js41637               http://steamcommunity.com/id/js41637   
4                js41637               http://steamcommunity.com/id/js41637   
...                  ...                                                ...   
59300  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
59301  76561198312638244  http://steamcommunity.com/profiles/76561198312...   
59302        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
59303        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   
59304        LydiaMorley           http://steamcommunity.com/id/LydiaMorley   

                          posted item_id  recommend

C:\Users\loren\AppData\Local\Temp\ipykernel_18864\846802356.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_user_reviews_sin_duplicados['year'] = new_df_user_reviews_sin_duplicados['posted'].apply(extract_year)
C:\Users\loren\AppData\Local\Temp\ipykernel_18864\846802356.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform th

In [136]:
new_df_user_reviews_sin_duplicados.dtypes
new_df_user_reviews_sin_duplicados.drop(columns=['posted'], inplace=True)

C:\Users\loren\AppData\Local\Temp\ipykernel_18864\3375310032.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_user_reviews_sin_duplicados.drop(columns=['posted'], inplace=True)


In [139]:
# Convertir la columna 'item_id' a tipo float
new_df_user_reviews_sin_duplicados['item_id'] = new_df_user_reviews_sin_duplicados['item_id'].astype(float)

C:\Users\loren\AppData\Local\Temp\ipykernel_18864\342199010.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_user_reviews_sin_duplicados['item_id'] = new_df_user_reviews_sin_duplicados['item_id'].astype(float)


In [140]:
new_df_user_reviews_sin_duplicados.dtypes

user_id       object
user_url      object
item_id      float64
recommend       bool
review        object
year           int32
dtype: object

In [141]:
table = pa.Table.from_pandas(new_df_user_reviews_sin_duplicados)
# Escribimos las tabla en un archivo Parquet
pq.write_table(table, 'Datasets/new_user_reviews.parquet')

Guardamos la nueva tabla en parquet para su futuro uso

Empezamos el último ETL, del json user_items

In [142]:
import gzip

# Lista para almacenar los datos del archivo JSON
rows = []

# Abrir el archivo comprimido en modo lectura de bytes ('rb')
with gzip.open(r"Datasets\users_items.json.gz", 'rb') as f:
    # Decodificar cada línea del archivo como texto UTF-8 y agregarla a la lista 'rows'
    for line in f:
        rows.append(line.decode('utf-8'))

# Crear un DataFrame de Pandas a partir de la lista de strings
df_user_items = pd.DataFrame(rows)

Corroboramos la información de nuestro nuevo Dataframe

In [143]:
df_user_items.head()
infoDF(df_user_items)

Información general del DataFrame:
Cantidad de filas: 88310
Cantidad de columnas: 1
Cantidad de filas con datos nulos: 0

Porcentaje de datos nulos por fila:
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
88305    0.0
88306    0.0
88307    0.0
88308    0.0
88309    0.0
Length: 88310, dtype: float64

Porcentaje de datos nulos por columna:
0    0.0
dtype: float64


Como podemos ver, aquí el error se encuentra en que existe únicamente una sola columna con toda la información anidada dentro.

In [144]:
print(df_user_items.columns)

RangeIndex(start=0, stop=1, step=1)


Comenzamos el proceso de desanidar todo nuestro dataframe para poder utilizarlo

In [145]:
# Crear una función para procesar cada fila del DataFrame original
def process_row(row):
    # Convertir el string de la columna 0 a un diccionario
    df_user_items = ast.literal_eval(row.iloc[0])
    
    # Extraer información
    user_id = df_user_items['user_id']
    items_count = df_user_items['items_count']
    steam_id = df_user_items['steam_id']
    user_url = df_user_items['user_url']
    
    # Crear una lista de diccionarios para los 'items'
    items_list = df_user_items['items']
    
    # Crear una lista para almacenar cada fila de datos
    rows = []
    
    # Iterar sobre los elementos de la lista de items
    for item in items_list:
        # Crear un diccionario con la información que necesitas
        row_df_user_items = {
            'user_id': user_id,
            'items_count': items_count,
            'steam_id': steam_id,
            'user_url': user_url,
            'item_id': item.get('item_id', None),  # Manejar casos donde el valor no está presente
            'item_name': item.get('item_name', None),
            'playtime_forever': item.get('playtime_forever', None),
            'playtime_2weeks': item.get('playtime_2weeks', None)
        }
        # Agregar el diccionario a la lista de filas
        rows.append(row_df_user_items)
    
    # Retornar la lista de filas
    return rows

# Aplicar la función a cada fila del DataFrame original y obtener una lista de listas de diccionarios
processed_data = df_user_items.apply(process_row, axis=1).tolist()

# Convertir la lista de listas de diccionarios en un solo DataFrame
new_df = pd.DataFrame([item for sublist in processed_data for item in sublist])

# Mostrar el nuevo DataFrame
print(new_df)

                   user_id  items_count           steam_id  \
0        76561197970982479          277  76561197970982479   
1        76561197970982479          277  76561197970982479   
2        76561197970982479          277  76561197970982479   
3        76561197970982479          277  76561197970982479   
4        76561197970982479          277  76561197970982479   
...                    ...          ...                ...   
5153204  76561198329548331            7  76561198329548331   
5153205  76561198329548331            7  76561198329548331   
5153206  76561198329548331            7  76561198329548331   
5153207  76561198329548331            7  76561198329548331   
5153208  76561198329548331            7  76561198329548331   

                                                  user_url item_id  \
0        http://steamcommunity.com/profiles/76561197970...      10   
1        http://steamcommunity.com/profiles/76561197970...      20   
2        http://steamcommunity.com/profiles/7

Ahora, podemos empezar a trabajar sobre nuestro dataframe

In [146]:
filas_duplicadas = new_df[new_df.duplicated()]
num_filas_duplicadas = len(filas_duplicadas)
if num_filas_duplicadas > 0:
    print("Se encontraron", num_filas_duplicadas, "filas duplicadas en el DataFrame.")
else:
    print("No se encontraron filas duplicadas en el DataFrame.")

Se encontraron 59104 filas duplicadas en el DataFrame.


Eliminamos los duplicados y vemos el tipo de datos que contiene

In [147]:
new_df_sin_duplicados = new_df.drop_duplicates()

In [148]:
new_df_sin_duplicados.dtypes

user_id             object
items_count          int64
steam_id            object
user_url            object
item_id             object
item_name           object
playtime_forever     int64
playtime_2weeks      int64
dtype: object

Para trabajar nuestros dataframes en conjunto, necesitamos que item_id sea tipo float y que item_name sea tipo string

In [149]:
# Convertir la columna 'item_id' a tipo float
new_df_sin_duplicados['item_id'] = new_df_sin_duplicados['item_id'].astype(float)

# Convertir la columna 'item_name' a tipo string
new_df_sin_duplicados['item_name'] = new_df_sin_duplicados['item_name'].astype(str)

C:\Users\loren\AppData\Local\Temp\ipykernel_18864\3657668227.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_sin_duplicados['item_id'] = new_df_sin_duplicados['item_id'].astype(float)
C:\Users\loren\AppData\Local\Temp\ipykernel_18864\3657668227.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_sin_duplicados['item_name'] = new_df_sin_duplicados['item_name'].astype(str)


Todo listo, podemos guardar nuestros dataframes en Parquet, y continuar el proyecto

In [150]:
table = pa.Table.from_pandas(new_df_sin_duplicados)
# Escribimos las tabla en un archivo Parquet
pq.write_table(table, 'Datasets/new_df_users_items.parquet')